encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
from NNutils import *
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
nbits = 4

In [3]:
linear_name = "etl1"
linear_name_full = "ex_linear1"
feedback_bits = 256
ncodebooks = 16 # max:64
ncentroids = 256
# if method == METHOD_MITHRAL:
#     ncentroids = 16
train_sam_num = 250 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    linearin_path_train= ''
    linearout_path_train= ''
    linearin_path_test = ''
    linearout_path_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_joined = '/data/hdr/transformer_data/joined'
    dir_train = os.path.join(dir_joined, 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join(dir_joined, 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    linearin_path_train= '%sin_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    y_train = '%s_y_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearout_path_train= '%sout_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
    linearin_path_test = '%sin_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
    linearout_path_test = '%sout_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
else:
    raise NameError("You are running the script in a new computer %s, please define dirs" % host_name)


weightpath = '%s_w_f%i.npy' % (linear_name_full, feedback_bits)
biaspath = '%s_b_f%i.npy' % (linear_name_full, feedback_bits)
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 


In [5]:
dataset_prepare(dir_joined, linear_name_full, feedback_bits, [train_sam_num, test_sam_num], batch_size, S1 = S1)

In [6]:
est3 = mm.estFactory(X_path=linearin_path_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits)

running method:  Mithral
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    6575.721211212877
learn_multisplits(): returning loss:  668.9090533033013
learn_multisplits(): initial loss:    9961.621111606059
learn_multisplits(): returning loss:  1161.2792363408953
learn_multisplits(): initial loss:    35586.14871557069
learn_multisplits(): returning loss:  3936.073377729709
learn_multisplits(): initial loss:    39788.90234938542
learn_multisplits(): returning loss:  3368.9217665083706
learn_multisplits(): initial loss:    12005.640384041488
learn_multisplits(): returning loss:  1209.5321292728186
learn_multisplits(): initial loss:    4902.752618937343
learn_multisplits(): returning loss:  638.4197876155376
learn_multisplits(): initial loss:    3260.960031088978
learn_multisplits(): returning loss:  407.60520566254854
learn_multisplits(): initial loss:    3863.0107275768896
learn_multisplits(): returning loss:  441.8834553062916
learn_multisplits(): initial l

/data/hdr/pq/bolt/experiments/python/matmul.py:325: UserWarning: Persisting input arguments took 0.61s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  est = _fitted_est_for_hparams(


In [7]:
x_test = np.load(dir_test+'/'+linearin_path_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last_re.astype(np.float32))
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)

[[-0.4266445  -0.44034454 -0.5346551  ... -0.31069577  1.1927899
  -0.23415726]
 [-0.4266445  -0.44034454 -0.5346551  ... -0.31069577  1.1927899
  -0.23415726]
 [-0.4266445  -0.44034454 -0.5346551  ... -0.31069577  1.1927899
  -0.23415726]
 ...
 [-0.17615527 -0.1898553  -0.2841658  ... -0.31069577  0.9423007
  -0.23415726]
 [-0.17615527 -0.1898553  -0.2841658  ... -0.06020654  0.9423007
  -0.23415726]
 [-0.17615527 -0.1898553  -0.03367659 ... -0.06020654  0.69181144
   0.01633197]]
y_out_last.shape:  (1024000, 512)
y_out_last_re.shape:  (1000, 32, 32, 512)
